# Sentiment Analysis with Trained Word Embeddings

In [7]:
import string
import pandas as pd
from string import punctuation
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

# Let's Define some Helper Functions

In [8]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# turn a doc into clean tokens
def clean_doc(doc, vocab):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # filter out tokens not in vocab
    tokens = [w for w in tokens if w in vocab]
    tokens = ' '.join(tokens)
    return tokens
 
# load all docs in a directory
def process_docs(directory, vocab, is_trian):
    documents = list()
    # walk through all files in the folder
    for filename in listdir(directory):
        # skip any reviews in the test set
        if is_trian and filename.startswith('cv9'):
            continue
        if not is_trian and not filename.startswith('cv9'):
            continue
        # create the full path of the file to open
        path = directory + '/' + filename
        # load the doc
        doc = load_doc(path)
        # clean doc
        tokens = clean_doc(doc, vocab)
        # add to list
        documents.append(tokens)
    return documents

# Read in our Vocabulary

In [9]:
# load the vocabulary
vocab_filename = 'data/movie_vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

# Let's prepare our training examples

### Note that we pass "True" to process_docs() so that we only return the training cases

In [5]:
# load training data
positive_lines = process_docs('data/txt_sentoken/pos', vocab, True)
negative_lines = process_docs('data/txt_sentoken/neg', vocab, True)
sentences = negative_lines + positive_lines
print('Total training sentences: %d' % len(sentences))

Total training sentences: 1800


# Fit tokenizer to our document set

### We will use the tokenizer to sequence our documents based on a word index of our vocabulary.

In [6]:
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(train_docs)
print(tokenizer.word_index)

NameError: name 'train_docs' is not defined

# Text Encoding

### We will first encode our text with our vocabulary word index we just created. 

In [6]:
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(train_docs)
print(encoded_docs[0])

[25, 16, 899, 3475, 46, 2103, 690, 3381, 1284, 14, 1164, 2, 377, 1149, 612, 1439, 18, 24, 4105, 445, 522, 136, 3, 9981, 94, 2000, 13894, 3, 899, 1297, 2068, 553, 256, 1454, 27, 3686, 56, 349, 5, 3476, 2, 1053, 92, 1175, 9143, 9, 409, 748, 6618, 754, 285, 222, 3563, 7910, 6, 27, 619, 149, 2484, 9, 1517, 137, 19605, 2, 6619, 138, 492, 159, 3382, 1133, 2756, 2273, 355, 3, 20, 233, 226, 199, 7005, 311, 1092, 19606, 1251, 59, 72, 990, 256, 445, 71, 914, 13, 402, 52, 273, 325, 79, 4, 273, 604, 13895, 16192, 815, 34, 4564, 1265, 77, 482, 199, 1872, 2608, 39, 218, 150, 9144, 1, 60, 147, 2683, 14, 152, 4251, 9, 776, 226, 476, 182, 61, 662, 3029, 51, 154, 3029, 200, 253, 419, 122, 21, 77, 326, 2883, 5, 1640, 5168, 22, 1176, 107, 4401, 5402, 1455, 4, 78, 3564, 1922, 113, 11016, 329, 21, 23, 151, 142, 50, 137, 21, 1, 326, 493, 1611, 229, 30, 4, 143, 21, 177, 72, 5, 173, 662, 19607, 1736, 59, 2069, 497, 838, 7006, 9145, 308, 114, 3935, 122, 350, 3, 1587, 3192, 924, 14, 26, 2806, 39, 38, 22, 320, 18

# Let's Prepare our Training Set

### We will also pad our sentences to a fixed size based on the maximium length document in our corpus.

In [8]:
# pad sequences
max_length = max([len(s.split()) for s in train_docs])
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# define training labels
ytrain = array([0 for _ in range(900)] + [1 for _ in range(900)])

# Let's Prepare our Test Set

### Note that we pass "False" to process_docs() so that we only return the test cases

In [9]:
# load all test reviews
positive_docs = process_docs('data/txt_sentoken/pos', vocab, False)
negative_docs = process_docs('data/txt_sentoken/neg', vocab, False)
test_docs = negative_docs + positive_docs
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(test_docs)
# pad sequences
Xtest = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# define test labels
ytest = array([0 for _ in range(100)] + [1 for _ in range(100)])

# Vocabulary Size

### Why are we adding 1 to our vocabulary size?

In [10]:
# define vocabulary size (largest integer value)
vocab_size = len(tokenizer.word_index) + 1

# Convolutional Neural Network Model

In [11]:
# define model
# The Sequential model is a linear stack of layers.
model = Sequential()
# the model will take as input an integer matrix of size (batch, input_length).
# the largest integer (i.e. word index) in the input should be no larger than (vocabulary size).
# now model.output_shape == (None, 1317, 100), where None is the batch dimension.
model.add(Embedding(vocab_size, 100, input_length=max_length))
# sentences are linear lists of words, so conv1d with a window size of 8 and 32 filters
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
# Downsample our input - helps with over-fitting
model.add(MaxPooling1D(pool_size=2))
# Flatten out our dimensions
model.add(Flatten())
# Feed into a fully connected layer
model.add(Dense(10, activation='relu'))
# Feed into a final neuron
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1317, 100)         2576800   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1310, 32)          25632     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 655, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 20960)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                209610    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 2,812,053
Trainable params: 2,812,053
Non-tra

# Compile and Train the model

### Adam Optimizer: Computes individual adaptive learning rates for different parameters from estimates of first and second moments of the gradients. 

In [12]:
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)

Epoch 1/10
 - 5s - loss: 0.6894 - acc: 0.5400
Epoch 2/10
 - 3s - loss: 0.5613 - acc: 0.7217
Epoch 3/10
 - 3s - loss: 0.1152 - acc: 0.9617
Epoch 4/10
 - 3s - loss: 0.0079 - acc: 0.9994
Epoch 5/10
 - 3s - loss: 0.0023 - acc: 1.0000
Epoch 6/10
 - 3s - loss: 0.0013 - acc: 1.0000
Epoch 7/10
 - 3s - loss: 9.5839e-04 - acc: 1.0000
Epoch 8/10
 - 3s - loss: 7.3775e-04 - acc: 1.0000
Epoch 9/10
 - 3s - loss: 5.9259e-04 - acc: 1.0000
Epoch 10/10
 - 3s - loss: 4.8831e-04 - acc: 1.0000


# Let's Test on our Holdout Set

In [13]:
# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 86.000000
